# Grouping and reshaping data

We're going to look at some different ways of grouping and aggregating data. We're building towards thinking about 'split', 'apply', and 'combine workflows, which look something like this:

![split-apply-combine](https://github.com/core-skills/02-getting-to-know-the-tools/blob/master/notebooks/split-apply-combine.png?raw=true)

(taken from Jake VanderPlas' excellent [Python data science handbook](https://github.com/jakevdp/PythonDataScienceHandbook) - check out all the notebooks available on github if you want more in-depth examples than what we've worked through today).

## Groupby

Find the pandas `groupby` method and work out how it works on your dataframe. Hint: try passing a categorical column from your data. 

If you don't have a categorical column but you do have a column of numbers, you can generate groups by binning the data into seperate bins using the `pandas.cut` function - something like this:

```python
import pandas
from numpy import inf
from random_data import random_dataframe

# Our faithful bogus dataframe
df = random_dataframe(30)

# Add a new column which bins the a values
df['how_big'] = pandas.cut(df.a, 
                           bins=[-inf, 50, inf],
                           labels=('low', 'high'))
```

`pandas.cut` can often be useful for investigating subsets of numerical data (e.g. ore grade in marginal blocks!).

In [3]:
import pandas

Next try to generate some summary statistics about each of your groups. The `info` and `describe` methods of the pandas dataframe are good places to start - try something like this:

```python
from random_data import random_dataframe

# Yet more bogus data
df = random_dataframe(100)

# Iterating in a for-loop
for category, grp_df in df.groupby('category'):
    print(f"\nInfo for group {category}")
    print(grp_df.describe())
```

In [8]:
from numpy import inf
from random_data import random_dataframe
df = random_dataframe(100)

In [10]:
from random_data import random_dataframe

In [11]:
df = random_dataframe(100)

In [18]:
def is_outlier(df):
    mean = df.a.mean()
    std = df.a.std()
    deviation = (df.a - mean) / std
    return deviation > 2

In [21]:
df[is_outlier(df)]

,category,a,b
26,y,128.557198,0.495028
74,z,122.668845,0.513140


In [22]:
is_outlier(df).sum()

2

In [23]:
for category, grp_df in df.groupby('category'):
    print(f"\nNumber of outliers for group {category}:", is_outlier(grp_df).sum())


Number of outliers for group x: 0

Number of outliers for group y: 1

Number of outliers for group z: 1


You don't have to iterate over the groups if you don't want to - you can pipeline them to an aggregating function directly (which is often easier to read).

```python
# Calculating an aggregation directly
df.groupby('category').sum()
```

Try looking at some of the other pandas aggregations: `count`, `first`, `last`, `mean`, `median`, `min`, `max`, `std`, `var`, `mad`, `prod`, `sum`. What do each of these do?

In [17]:
df.groupby('category').describe()

a                                                         \
         count       mean        std        min        25%        50%   
category                                                                
x         25.0  49.604467  29.959781   7.722178  23.700891  44.385541   
y         37.0  60.333224  30.751280  10.312657  34.454169  61.475491   
z         38.0  43.976325  33.178138  -0.129281  15.461963  31.412148   

                                    b                                          \
                75%         max count      mean       std       min       25%   
category                                                                        
x         75.330321  109.444158  25.0  0.463923  0.270657  0.037193  0.267520   
y         87.092913  128.557198  37.0  0.525191  0.308010  0.013082  0.257111   
z         73.861787  122.668845  38.0  0.472831  0.300223  0.003826  0.239229   

                                        
               50%       75%       max  
category                                
x         0.441681  0.716652  0.865592  
y         0.495028  0.796725  0.994678  
z         0.470424  0.687353  0.984340

In [14]:
df.groupby('category').mean()

,a,b
category,,
x,49.604467,0.463923
y,60.333224,0.525191
z,43.976325,0.472831


In [15]:
df.groupby('category').max()

,a,b
category,,
x,109.444158,0.865592
y,128.557198,0.994678
z,122.668845,0.984340


In [16]:
df.groupby('category').min()

,a,b
category,,
x,7.722178,0.037193
y,10.312657,0.013082
z,-0.129281,0.003826


Take a look at some of the more advanced group options - for example you can set a category as an index, and pass functions which take an index and output a group.

```python
# Calculating an aggregation by specifying a mapping 
# from index to group
mapping = {'x': 'first', 'y': 'first', 'z': 'second'}
df.set_index('category').groupby(mapping).mean()
```

How might you write a small function to start to aggregate or summarize the data in your data's groups in more complex ways?

In [24]:
mapping = {'x': 'first', 'y': 'first', 'z': 'second'}

In [25]:
df.set_index('category').groupby(mapping).mean()

,a,b
first,56.007112,0.500486
second,43.976325,0.472831


In [26]:
df.set_index('category').groupby(mapping).sum()

,a,b
first,3472.440956,31.030147
second,1671.100343,17.967578


In [27]:
df.set_index('category').groupby(mapping).describe()

a                                                                   \
       count       mean        std       min        25%        50%        75%   
first   62.0  56.007112  30.649397  7.722178  31.440855  56.852935  84.240791   
second  38.0  43.976325  33.178138 -0.129281  15.461963  31.412148  73.861787   

                       b                                                    \
               max count      mean       std       min       25%       50%   
first   128.557198  62.0  0.500486  0.292795  0.013082  0.259713  0.463746   
second  122.668845  38.0  0.472831  0.300223  0.003826  0.239229  0.470424   

                            
             75%       max  
first   0.782776  0.994678  
second  0.687353  0.984340

In [28]:
df

,category,a,b
0,z,21.739758,0.125521
1,x,23.700891,0.037193
2,x,57.570650,0.799073
3,z,85.163590,0.003826
4,x,59.828485,0.585415
5,y,87.092913,0.767884
6,y,90.781288,0.711614
7,z,85.163451,0.627954
8,x,7.722178,0.865592
9,z,59.708576,0.725058


In [29]:
pivot

NameError: name 'pivot' is not defined

## Pivot table

Pivot tables are a lot like groupby operations but instead of ending up with one column of groups we can end up with multidimensional arrays of aggregations. 

In the diagram at the top of the page, you can think of a pivot table splitting the data using more than one column in the 'split' step.

This is generally more useful when we want to start to aggregate along multiple dimensions.

Using our example from above:

```python
import pandas
from numpy import inf
from random_data import random_dataframe

# Our faithful bogus dataframe
df = random_dataframe(3000, categories='uvwxyz')

# Add a new column which bins the a values
df['how_big'] = pandas.cut(df.a, 
                           bins=[-inf, 50, inf],
                           labels=('low', 'high'))

# make a new pivot table that calculates the mean for each
# of our subcategories - both 'x,y,z' and 'low' and 'high'
pivot = df.pivot_table('b', index='category', columns='how_big', aggfunc='mean')
```

Try creating a pivot table on your own data (as before you can use `pandas.cut` to bin numerical data if that's more useful).

In [ ]:
import pandas
from numpy import inf
from random_data import random_dataframe

# Our faithful bogus dataframe
df = random_dataframe(3000, categories='uvwxyz')

# Add a new column which bins the a values
df['how_big'] = pandas.cut(df.a, 
                           bins=[-inf, 25, 50, 75, inf],
                           labels=('tiny', 'small', 'medium' 'large', 'huge'))

# make a new pivot table that calculates the mean for each
# of our subcategories - both 'x,y,z' and 'low' and 'high'
pivot = df.pivot_table('b', index='category', columns='how_big', aggfunc='sum')

In [30]:
import pandas

In [31]:
from numpy import inf

In [32]:
from random_data import random_dataframe

In [34]:
df = random_dataframe(3000, categories='uvwxyz')

In [36]:
df['how_big'] = pandas.cut(df.a, 
                           bins=[-inf, 25, 50, 75, inf],
                           labels=('tiny', 'small', 'medium' 'large', 'huge'))


In [37]:
pivot = df.pivot_table('b', index='category', columns='how_big', aggfunc='sum')

In [38]:
df.groupby('how_big').count()

,category,a,b
how_big,,,
tiny,758,758,758
small,762,762,762
mediumlarge,756,756,756
huge,724,724,724


## Plotting data

Next we're going to use [seaborn](seaborn.pydata.org) to generate some pretty plots of our data. 

Most Python tutorials will introduce [matplotlib](https://matplotlib.org) at this stage because it's the default but seaborn is a much higher-level library with a nicer API, especially for exploratory vis (matplotlib will probably make more sense to you if you're coming from MATLAB world though). The only hangover is that we need to include the `%matplotlib inline` cell magic to tell Jupyter to render the graphics inline for us. 

We'll start by looking at our random dataset.  

In [ ]:
%matplotlib inline
import seaborn
seaborn.set()

from random_data import random_dataframe

# Set up our dataframe and pivot table
df = random_dataframe(3000, categories='uvwxyz')
df['how_big'] = pandas.cut(df.a, 
                           bins=[-inf, 25, 50, 75, inf],
                           labels=('tiny', 'small', 'medium' 'large', 'huge'))
df.head()

For one-dimensional dataset we can try `seaborn.distplot`, `seaborn.kdeplot` and `seaborn.rugplot` to visualize the data.

In [ ]:
seaborn.distplot(df.a)

In [ ]:
seaborn.kdeplot(df.a)

We can also use `jointplot` to generate a scatter and histograms of sets of data

In [ ]:
seaborn.jointplot('a', 'b', df)

In [ ]:
seaborn.jointplot('a', 'b', df, kind='hex')

For times when you want 'plot everything against everything else' you can do something like

In [ ]:
seaborn.pairplot(df)

This is really useful for pulling out relationships between variables

In [ ]:
df['c'] = df.a + df.b * df.a
df['d'] = df.c * df.b + df.a

In [ ]:
seaborn.pairplot(df)

In [ ]:
g = seaborn.PairGrid(df)
g.map_diag(seaborn.kdeplot)
g.map_offdiag(seaborn.kdeplot, n_levels=6);

Seaborn also has a heap of support for categorical data. We can also include more dimensions in the visualization by specifying further dimensions as colors or point size

In [ ]:
seaborn.stripplot('a', 'how_big', hue='category', data=df, jitter=True, dodge=True)

We can also visualize the pivot table we generated above with heatmaps. 

In [ ]:
pivot = df.pivot_table('b', index='category', columns='how_big', aggfunc='sum')
pivot

In [ ]:
seaborn.heatmap(pivot)

In [ ]:
seaborn.clustermap(pivot)

Seaborn can get a lot more complicated than this and it's worth digging through the examples to find useful ways of slicing and dicing your dataframes into pictures.

Now try this out on your own dataset!